### ReAct Agent
ReAct agent is an AI agent that follows the Reasoning and Acting (ReAct) framework, which combines chain-of-thought (CoT) reasoning with external tool use. This approach allows AI models to dynamically plan, execute, and adapt their actions based on new information, rather than following predefined workflows

In [1]:
import os
import json
import requests
from openai import AzureOpenAI
from bs4 import BeautifulSoup
import urllib.parse
from dotenv import load_dotenv
import datetime
import pytz

load_dotenv()

# Set up Azure OpenAI client
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION")
deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4.1-mini")

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

# Define a simple web search tool using DuckDuckGo without API key
def web_search(query: str) -> str:
    """Search the web for information using DuckDuckGo."""
    try:
        # URL encode the query for use in the URL
        encoded_query = urllib.parse.quote_plus(query)
        
        # Create headers to simulate a browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # Make request to DuckDuckGo HTML search
        url = f"https://html.duckduckgo.com/html/?q={encoded_query}"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            return f"Search failed with status code {response.status_code}"
            
        # Parse the HTML response
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract search results
        results = []
        result_elements = soup.select('.result')[:3]  # Get first 3 results
        
        if not result_elements:
            return "No search results found or page format has changed."
            
        for result in result_elements:
            title_elem = result.select_one('.result__title')
            snippet_elem = result.select_one('.result__snippet')
            link_elem = result.select_one('.result__url')
            
            title = title_elem.get_text(strip=True) if title_elem else "No title"
            snippet = snippet_elem.get_text(strip=True) if snippet_elem else "No description"
            
            # Extract href and clean it
            if link_elem:
                link = link_elem.get_text(strip=True)
            else:
                link_anchor = title_elem.select_one('a') if title_elem else None
                link = link_anchor.get('href', "No link") if link_anchor else "No link"
                # DuckDuckGo sometimes uses redirects
                if link.startswith('/'):
                    link = "https://duckduckgo.com" + link
            
            results.append(f"Title: {title}\nSnippet: {snippet}\nLink: {link}")
            
        return "\n\n".join(results) if results else "No search results found."
        
    except Exception as e:
        # Return a fallback for testing
        return f"Error during web search: {str(e)}. This is a mock search result for query: {query}"

# Define a simple calculator tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Be careful with eval() in production - this is for demonstration
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

def weather_search(location: str) -> str:
    """Get current weather information for a location using Open-Meteo API."""
    try:
        # Use Open-Meteo API which doesn't require an API key
        # First, geocode the location to get coordinates
        geocoding_url = f"https://geocoding-api.open-meteo.com/v1/search"
        geo_response = requests.get(
            geocoding_url,
            params={"name": location, "count": 1}
        )
        
        if geo_response.status_code != 200:
            return f"Location search failed with status code {geo_response.status_code}"
            
        geo_data = geo_response.json()
        if not geo_data.get("results"):
            return f"Could not find location: {location}"
            
        # Get the latitude and longitude
        lat = geo_data["results"][0]["latitude"]
        lon = geo_data["results"][0]["longitude"]
        place_name = geo_data["results"][0]["name"]
        country = geo_data["results"][0].get("country", "")
        
        # Now get the weather data
        weather_url = "https://api.open-meteo.com/v1/forecast"
        weather_response = requests.get(
            weather_url,
            params={
                "latitude": lat,
                "longitude": lon,
                "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
                "timezone": "auto"
            }
        )
        
        if weather_response.status_code != 200:
            return f"Weather search failed with status code {weather_response.status_code}"
            
        weather_data = weather_response.json()
        current = weather_data.get("current", {})
        
        # Map weather codes to descriptions
        weather_descriptions = {
            0: "Clear sky",
            1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
            45: "Fog", 48: "Depositing rime fog",
            51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
            61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
            71: "Slight snow fall", 73: "Moderate snow fall", 75: "Heavy snow fall",
            80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
            95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail"
        }
        
        weather_code = current.get("weather_code")
        weather_desc = weather_descriptions.get(weather_code, "Unknown")
        
        result = f"Weather for {place_name}, {country}:\n"
        result += f"Temperature: {current.get('temperature_2m')}°{weather_data.get('current_units', {}).get('temperature_2m', 'C')}\n"
        result += f"Humidity: {current.get('relative_humidity_2m')}%\n"
        result += f"Conditions: {weather_desc}\n"
        result += f"Wind Speed: {current.get('wind_speed_10m')} {weather_data.get('current_units', {}).get('wind_speed_10m', 'km/h')}"
        
        return result
    except Exception as e:
        return f"Error getting weather information: {str(e)}"

# Let's implement a new tool for getting the current time
def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in a specified timezone."""
    try:
        # Check if the provided timezone is valid
        if timezone not in pytz.all_timezones:
            # Try to find close matches
            similar_timezones = [tz for tz in pytz.all_timezones if timezone.lower() in tz.lower()]
            
            if similar_timezones:
                # Use the first similar timezone
                timezone = similar_timezones[0]
                note = f" (using {timezone} as specified timezone wasn't found exactly)"
            else:
                # Default to UTC if no match
                timezone = "UTC"
                note = " (defaulting to UTC as specified timezone wasn't found)"
        else:
            note = ""
            
        # Get the current time in the specified timezone
        tz = pytz.timezone(timezone)
        current_time = datetime.datetime.now(tz)
        
        # Format the time string
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S %Z")
        return f"Current time in {timezone}: {formatted_time}{note}"
    
    except Exception as e:
        return f"Error getting current time: {str(e)}"

# Define tools for Azure OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for current information on a topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "weather_search",
            "description": "Get the current weather for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city or location to get weather for"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a specified timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "The timezone to get the current time for (e.g., 'UTC', 'America/New_York', 'Europe/London', 'Asia/Tokyo')"
                    }
                },
                "required": []
            }
        }
    }
]

# Define a mapping from tool names to functions
tool_mapping = {
    "web_search": web_search,
    "calculator": calculator,
    "weather_search": weather_search,
    "get_current_time": get_current_time
}

class ReActAgent:
    def __init__(self, client, model, tools, tool_mapping):
        self.client = client
        self.model = model
        self.tools = tools
        self.tool_mapping = tool_mapping
        self.conversation_history = []
        
    def call_tool(self, tool_call):
        """Execute a tool call and return the result."""
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        if function_name in self.tool_mapping:
            function = self.tool_mapping[function_name]
            return function(**function_args)
        else:
            return f"Tool {function_name} not found."
    
    def run(self, user_input, max_turns=5):
        """Run the ReAct agent loop."""
        # Create a system message that instructs the model to follow the ReAct pattern
        system_message = {
            "role": "system",
            "content": """You are a helpful assistant that follows the ReAct (Reasoning + Acting) framework to solve tasks.
When approaching a problem, follow these steps:
1. Thought: Think step-by-step about how to solve the problem and what tools might help.
2. Action: Choose a tool to use and provide the necessary arguments.
3. Observation: After seeing the result of your action, analyze what you learned.
4. Repeat the Thought-Action-Observation cycle until you can provide a complete answer.

Always start with a "Thought:" before deciding on an action. After receiving an observation, start again with "Thought:".
Remember that your thoughts are visible to the user, so make them clear and logical."""
        }
        
        # Initialize conversation with system message
        if not any(msg.get("role") == "system" for msg in self.conversation_history):
            self.conversation_history.insert(0, system_message)
        
        self.conversation_history.append({
            "role": "user",
            "content": user_input
        })
        
        for turn in range(max_turns):
            print(f"\n--- Turn {turn + 1} ---")
            
            # Get model response
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                tools=self.tools,
                tool_choice="auto"
            )
            
            assistant_message = response.choices[0].message
            
            # Add assistant's response to conversation history
            # We need to capture both content and tool_calls in the assistant message
            assistant_msg = {"role": "assistant"}
            if assistant_message.content is not None:
                assistant_msg["content"] = assistant_message.content
            if hasattr(assistant_message, 'tool_calls') and assistant_message.tool_calls:
                assistant_msg["tool_calls"] = [
                    {
                        "id": tool_call.id,
                        "type": "function", 
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments
                        }
                    } 
                    for tool_call in assistant_message.tool_calls
                ]
            
            # Add the properly structured assistant message
            self.conversation_history.append(assistant_msg)
            
            # Print the assistant's thought process
            thought_content = assistant_message.content or 'No specific thought provided.'
            print(f"Thought: {thought_content}")
            
            # Check for tool calls
            if hasattr(assistant_message, 'tool_calls') and assistant_message.tool_calls:
                for tool_call in assistant_message.tool_calls:
                    # Extract and print the action
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    print(f"Action: {function_name}({function_args})")
                    
                    # Execute the tool
                    result = self.call_tool(tool_call)
                    print(f"Observation: {result}")
                    
                    # Add the tool result to the conversation with proper format
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": result
                    })
            else:
                # If no tool calls, the agent has reached a conclusion
                print("Agent has completed reasoning.")
                break
        
        # Return the final response
        return self.conversation_history[-1]["content"]


In [6]:

# Initialize the ReAct agent
agent = ReActAgent(client, deployment_name, tools, tool_mapping)  
query = "Where was the Microsoft founder was born? And how old is he/she now?"
result = agent.run(query)
print("\n--- Final Answer ---")
print(result)


--- Turn 1 ---
Thought: Thought: The Microsoft founder is Bill Gates. I need to find out where he was born and his current age. For the age, I will need his birthdate and the current date.

First, I will search for Bill Gates' birthdate and place of birth. Then I will calculate his age based on the current date.
Action: web_search({'query': 'Bill Gates birthdate and place of birth'})
Thought: Thought: The Microsoft founder is Bill Gates. I need to find out where he was born and his current age. For the age, I will need his birthdate and the current date.

First, I will search for Bill Gates' birthdate and place of birth. Then I will calculate his age based on the current date.
Action: web_search({'query': 'Bill Gates birthdate and place of birth'})
Observation: Title: Find Date Of Birth - Top Free Genealogy SiteAdViewing ads is privacy protected by DuckDuckGo. Ad clicks are managed by Microsoft's ad network (more info).
Snippet: The easy & exciting way to explore your family history. 